In [1]:
#LOAD: libararies, parameters, functions
from cmath import nan
from datetime import date
#import nasdaqdatalink
#nasdaqdatalink.ApiConfig.api_key = "D2S-LBYJrbd-1mNzGRGg"
import copy


import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
from   plotly.subplots import make_subplots
import nasdaqdatalink
from sklearn.preprocessing import MinMaxScaler

plt.rcParams["figure.figsize"] = (10, 2)

from Quantapp.EconomicData import EconomicData

ed = EconomicData()

leading_indicators    = ed.get_leading_indicators()
coincident_indicators = ed.get_coincident_indicators()
lagging_indictators   = ed.get_lagging_indicators()
housing_market_data    = ed.get_housing_market_data()



def growth_rate(arr):
    return pd.Series(np.exp(np.diff(np.log(arr))) - 1)



def plot_with_mean_and_std(data_df, title='Series Plot', std_devs=None):
    fig = go.Figure()
    
    # Define colors and line styles
    line_style = 'dash'
    colors = ['deepskyblue', 'orange', 'red', 'magenta', 'green']
    
    if std_devs is None:
        std_devs = [1, 2, 2.5]
    
    # Plot individual series
    for i, column in enumerate(data_df.columns):
        series = data_df[column]
        fig.add_trace(go.Scatter(x=series.index, y=series, mode='lines', name=column, line=dict(color=colors[i % len(colors)])))
    
    # Compute the average series across all columns
    average_series = data_df.mean(axis=1)
    
    # Compute mean and standard deviations of the average series
    avg_mean = average_series.mean()
    avg_std = average_series.std()
    
    # Mean line for average series
    fig.add_shape(type='line', x0=average_series.index.min(), x1=average_series.index.max(), y0=avg_mean, y1=avg_mean, line=dict(color='yellow', width=2), name='Average Mean')
    fig.add_annotation(x=average_series.index.max(), y=avg_mean, text=f'Average Mean: {avg_mean:.2f}', showarrow=False, yshift=10, font=dict(color='yellow'))
    
    # Standard deviation lines for average series
    for sd in std_devs:
        line_color = 'cyan'
        
        # Positive standard deviation lines
        fig.add_shape(type='line', x0=average_series.index.min(), x1=average_series.index.max(), y0=avg_mean + sd * avg_std, y1=avg_mean + sd * avg_std, line=dict(color=line_color, dash=line_style, width=1), name=f'Average +{sd} Std Dev')
        fig.add_annotation(x=average_series.index.max(), y=avg_mean + sd * avg_std, text=f'+{sd} Std Dev', showarrow=False, yshift=10, font=dict(color=line_color))
        
        # Negative standard deviation lines
        fig.add_shape(type='line', x0=average_series.index.min(), x1=average_series.index.max(), y0=avg_mean - sd * avg_std, y1=avg_mean - sd * avg_std, line=dict(color=line_color, dash=line_style, width=1), name=f'Average -{sd} Std Dev')
        fig.add_annotation(x=average_series.index.max(), y=avg_mean - sd * avg_std, text=f'-{sd} Std Dev', showarrow=False, yshift=10, font=dict(color=line_color))
    
    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title='Value',
        template='plotly_dark',
        height=600,  # Adjust this value as needed to increase the height
        legend=dict(x=0.8, y=0.9)  # Adjust legend position to avoid overlap
    )
    
    return fig

def get_recession_periods(df):
    recession_periods = []
    recession_start = None

    for current_date, value in df['Value'].items():
        if value == 1 and recession_start is None:
            recession_start = current_date
        elif value == 0 and recession_start is not None:
            recession_periods.append((recession_start, current_date - pd.DateOffset(days=1)))
            recession_start = None

    # Handle case where recession ends at the last date
    if recession_start is not None:
        recession_periods.append((recession_start, df.index[-1]))

    return recession_periods

def highlight_recession_periods(fig, recession_periods, min_price, max_price):
    # Make a deep copy of the figure
    
    # Add recession periods to the copied figure
    for start, end in recession_periods:
        fig.add_trace(go.Scatter(
            x=[start, end, end, start, start],
            y=[min_price, min_price, max_price, max_price, min_price],
            fill='toself',
            fillcolor='rgba(255,0,0,0.2)',
            line=dict(color='rgba(255,0,0,0)'),
            name='Recession',
            showlegend=False
        ))
    return fig



DataLinkError: (Status 403) Something went wrong. Please try again. If you continue to have problems, please contact us at connect@data.nasdaq.com.

In [1]:

# Retrieve CPI and PPI data
cpi_data = ed.get_inflation_data()['CPI']
ppi_data = ed.get_inflation_data()['PPI']

# Ensure both data sets have timezone-naive date indexes (if needed)
cpi_data.index = cpi_data.index.tz_localize(None)
ppi_data.index = ppi_data.index.tz_localize(None)

# Combine CPI and PPI into a single DataFrame
combined_data = pd.concat([cpi_data, ppi_data], axis=1)
combined_data.columns = ['CPI', 'PPI']

# Dropping any NaN values
combined_data = combined_data.dropna()

# Calculate CPI minus PPI
combined_data['CPI minus PPI'] = combined_data['CPI'] - combined_data['PPI']

# Dropping any NaN values resulting from the percentage change calculation
combined_data = combined_data.dropna()

# Calculate rolling correlation (12-month window)
rolling_corr = combined_data['CPI'].rolling(window=12).corr(combined_data['PPI'])
fig = building_permits_minus_housing_starts_fig = highlight_recession_periods(fig, recession_periods, min_price, max_price)

# Plot 1: CPI and PPI Over Time
fig_cpi_ppi = go.Figure()
fig_cpi_ppi.add_trace(go.Scatter(x=combined_data.index, y=combined_data['CPI'], name='CPI'))
fig_cpi_ppi.add_trace(go.Scatter(x=combined_data.index, y=combined_data['PPI'], name='PPI'))
fig_cpi_ppi.update_layout(title='CPI and PPI Over Time', xaxis_title='Date', yaxis_title='Value')
fig_cpi_ppi = highlight_recession_periods(fig_cpi_ppi, recession_periods, combined_data['CPI'].min(), combined_data['CPI'].max())

fig_cpi_ppi.show()

# Plot 2: CPI minus PPI Over Time
fig_cpi_minus_ppi = go.Figure()
fig_cpi_minus_ppi.add_trace(go.Scatter(x=combined_data.index, y=combined_data['CPI minus PPI'], name='CPI - PPI'))
fig_cpi_minus_ppi.update_layout(title='CPI - PPI Over Time', xaxis_title='Date', yaxis_title='Difference')
fig_cpi_minus_ppi = highlight_recession_periods(fig_cpi_minus_ppi, recession_periods, combined_data['CPI minus PPI'].min(), combined_data['CPI minus PPI'].max())
fig_cpi_minus_ppi.show()

# Plot 3: Rolling Correlation Between CPI and PPI
fig_rolling_corr = go.Figure()
fig_rolling_corr.add_trace(go.Scatter(x=combined_data.index, y=rolling_corr, name='Rolling Correlation (12 months)'))
fig_rolling_corr.update_layout(title='Rolling Correlation (12 months)', xaxis_title='Date', yaxis_title='Correlation')
fig_rolling_corr.show()


NameError: name 'ed' is not defined

In [ ]:
import pandas as pd
import plotly.graph_objects as go
recession_periods = get_recession_periods(recession_ranges)

# Retrieve GDP and inflation data
gdp_data = ed.get_gdp_data()
inflation_data = ed.get_inflation_data()
nber_ranges = recession_indicators['NBER based']

# Calculate Output Gap
output_gap = 100 * (gdp_data['real gdp'] - gdp_data['real potential gdp']) / gdp_data['real potential gdp']

# Create Output Gap Plot
fig_output_gap = go.Figure()
fig_output_gap.add_trace(go.Scatter(x=output_gap.index, y=output_gap['Value'], mode='lines', name='Output Gap'))
fig_output_gap.add_hline(y=0, line_dash="dash", line_color="#000000", line_width=1)
fig_output_gap.update_layout(title='Output Gap', xaxis_title='Date', yaxis_title='Output Gap (%)')
fig_output_gap = highlight_recession_periods(fig_output_gap, recession_periods, output_gap['Value'].min(), output_gap['Value'].max())
fig_output_gap.show()

# Calculate PPI - CPI Difference
ppi_cpi_difference = pd.DataFrame()
ppi_cpi_difference['3_month_pct_change'] = inflation_data['PPI'].pct_change(3) - inflation_data['CPI'].pct_change(3)
ppi_cpi_difference['6_month_pct_change'] = inflation_data['PPI'].pct_change(6) - inflation_data['CPI'].pct_change(6)
ppi_cpi_difference['12_month_pct_change'] = inflation_data['PPI'].pct_change(12) - inflation_data['CPI'].pct_change(12)

# Slice the last 5 years (60 months)
ppi_cpi_difference = ppi_cpi_difference.iloc[-60*5:]

# Create PPI - CPI Difference Plot
fig_ppi_cpi_difference = go.Figure()
fig_ppi_cpi_difference.add_trace(go.Scatter(x=ppi_cpi_difference.index, y=ppi_cpi_difference['3_month_pct_change'], mode='lines', name='3-Month % Change'))
fig_ppi_cpi_difference.add_trace(go.Scatter(x=ppi_cpi_difference.index, y=ppi_cpi_difference['6_month_pct_change'], mode='lines', name='6-Month % Change'))
fig_ppi_cpi_difference.add_trace(go.Scatter(x=ppi_cpi_difference.index, y=ppi_cpi_difference['12_month_pct_change'], mode='lines', name='12-Month % Change'))
fig_ppi_cpi_difference.add_hline(y=0, line_dash="dash", line_color="#000000", line_width=1)
fig_ppi_cpi_difference.update_layout(title='PPI - CPI Difference', xaxis_title='Date', yaxis_title='Difference')
fig_ppi_cpi_difference = highlight_recession_periods(fig_ppi_cpi_difference, recession_periods, ppi_cpi_difference.min().min(), ppi_cpi_difference.max().max())
fig_ppi_cpi_difference.show()


# Plot Sticky Price CPI with plotly
fig_sticky_cpi = go.Figure()
fig_sticky_cpi.add_trace(go.Scatter(x=inflation_data['Sticky Price Consumer Price Index'].index, y=inflation_data['Sticky Price Consumer Price Index']['Value'], mode='lines', name='Sticky Price CPI'))
fig_sticky_cpi.update_layout(title='Sticky Price Consumer Price Index', xaxis_title='Date', yaxis_title='Value')
fig_sticky_cpi.show()


In [ ]:
#Housing Market Data
import yfinance as yf

# Extract Series from DataFrames
building_permits_series = housing_market_data['Building Permits'].iloc[:, 0]
housing_starts_series = housing_market_data['Housing Starts'].iloc[:, 0]
new_home_sales_series = housing_market_data['New Home Sales'].iloc[:, 0]

# Align indexes
common_index = building_permits_series.index.intersection(housing_starts_series.index).intersection(new_home_sales_series.index)

# Reindex Series
building_permits_series = building_permits_series.reindex(common_index)
housing_starts_series = housing_starts_series.reindex(common_index)
new_home_sales_series = new_home_sales_series.reindex(common_index)

# Combine into DataFrame
combined_df = pd.DataFrame({
    'Building Permits': building_permits_series,
    'Housing Starts': housing_starts_series,
    'New Home Sales': new_home_sales_series
})
# Plot original data

fig = plot_with_mean_and_std(combined_df.copy(), title='Housing Market Data')
recession_ranges = recession_indicators['NBER based'].reindex(common_index)
recession_periods = get_recession_periods(recession_ranges)
min_price = combined_df.min().min()
max_price = combined_df.max().max()

housing_market_data_fig = highlight_recession_periods(fig, recession_periods, min_price, max_price)
housing_market_data_fig.show()

# Plot building_permits_series minus housing_starts_series
building_permits_minus_housing_starts = building_permits_series - housing_starts_series
moving_average = building_permits_minus_housing_starts.rolling(window=12).mean()

fig = go.Figure()
fig.add_trace(go.Scatter(x=building_permits_minus_housing_starts.index, y=building_permits_minus_housing_starts, mode='lines', name='Building Permits - Housing Starts'))
fig.add_trace(go.Scatter(x=moving_average.index, y=moving_average, mode='lines', name='Moving Average'))
fig.add_trace(go.Scatter(x=building_permits_minus_housing_starts.index, y=[0] * len(building_permits_minus_housing_starts), mode='lines', name='Zero', line=dict(color='gray', dash='dash')))
fig.update_layout(title='Building Permits - Housing Starts', xaxis_title='Date', yaxis_title='Difference', template='plotly_dark')
min_price = building_permits_minus_housing_starts.min()
max_price = building_permits_minus_housing_starts.max()
fig = building_permits_minus_housing_starts_fig = highlight_recession_periods(fig, recession_periods, min_price, max_price)
fig.show()

# plot volatility of building permits minus housing starts
building_permits_minus_housing_starts_volatility = building_permits_minus_housing_starts.rolling(window=12).std()
fig = plot_with_mean_and_std(building_permits_minus_housing_starts_volatility.to_frame(), title='Building Permits - Housing Starts Volatility')
fig.update_layout(xaxis_title='Date', yaxis_title='Volatility')





# Add recession periods
fig = highlight_recession_periods(fig, recession_periods, building_permits_minus_housing_starts_volatility.min(), building_permits_minus_housing_starts_volatility.max())
fig.show()

# plot buuilding permits minus new home sales
building_permits_minus_new_home_sales = building_permits_series - new_home_sales_series
fig = plot_with_mean_and_std(building_permits_minus_new_home_sales.to_frame(), title='Building Permits - New Home Sales')
fig.update_layout(xaxis_title='Date', yaxis_title='Difference')



# Add recession periods
fig = highlight_recession_periods(fig, recession_periods, building_permits_minus_new_home_sales.min(), building_permits_minus_new_home_sales.max())

# plot housing starts minus new home sales
housing_starts_minus_new_home_sales = housing_starts_series - new_home_sales_series
fig2 = plot_with_mean_and_std(housing_starts_minus_new_home_sales.to_frame(), title='Housing Starts - New Home Sales')
fig2.update_layout(xaxis_title='Date', yaxis_title='Difference')

# Add recession periods
fig2 = highlight_recession_periods(fig2, recession_periods, housing_starts_minus_new_home_sales.min(), housing_starts_minus_new_home_sales.max())

# Combine the two figures
fig.add_traces(fig2.data)

fig.show()

# Get stock data for VNQ
vnq = yf.Ticker("VNQ")
vnq_data = vnq.history(period="max")


#plot xhb which is a home construction etf
xhb = yf.Ticker("XHB")
xhb_data = xhb.history(period="max")

#plot rez which is a residential real estate etf
rez = yf.Ticker("REZ")
rez_data = rez.history(period="max")



# Plot vnq - xhb
vnq_returns = vnq_data['Close'].pct_change(12)
xhb_returns = xhb_data['Close'].pct_change(12)
returns_1y = vnq_returns - xhb_returns

fig = plot_with_mean_and_std(returns_1y.to_frame(), title='Difference in VNQ and XHB 1-Year Returns')
fig.show()

# Plot the 5 year returns for vnq and xhb instead. on the same graph
vnq_returns = vnq_data['Close'].pct_change(60)
xhb_returns = xhb_data['Close'].pct_change(60)

return_5y = vnq_returns - xhb_returns

fig = plot_with_mean_and_std(return_5y.to_frame(), title='Difference in VNQ and XHB 5-Year Returns')
fig.show()

# Plot the 10 year returns for vnq and xhb instead. on the same graph
vnq_returns = vnq_data['Close'].pct_change(120)
xhb_returns = xhb_data['Close'].pct_change(120)

return_10y = vnq_returns - xhb_returns



# Plot vnq - rez
vnq_returns = vnq_data['Close'].pct_change(12)
rez_returns = rez_data['Close'].pct_change(12)
returns_1y = vnq_returns - rez_returns

fig = plot_with_mean_and_std(returns_1y.to_frame(), title='Difference in VNQ and REZ 1-Year Returns')
fig.show()

# Plot the 5 year returns for vnq and rez instead. on the same graph
vnq_returns = vnq_data['Close'].pct_change(60)
rez_returns = rez_data['Close'].pct_change(60)

return_5y = vnq_returns - rez_returns

fig = plot_with_mean_and_std(return_5y.to_frame(), title='Difference in VNQ and REZ 5-Year Returns')
fig.show()

# Plot the 10 year returns for vnq and rez instead. on the same graph
vnq_returns = vnq_data['Close'].pct_change(120)
rez_returns = rez_data['Close'].pct_change(120)

return_10y = vnq_returns - rez_returns

fig = plot_with_mean_and_std(return_10y.to_frame(), title='Difference in VNQ and REZ 10-Year Returns')
fig.show()

fig = plot_with_mean_and_std(return_10y.to_frame(), title='Difference in VNQ and XHB 10-Year Returns')
fig.show()


In [ ]:
import plotly.graph_objects as go
import pandas as pd
ism_new_order_index = leading_indicators['ism_new_order_index']['Index']
average_weekly_hours_manufacturing = leading_indicators['average_weekly_hours_manufacturing']
initial_claims = leading_indicators['initial_claims']
leading_credit_index = leading_indicators['leading_credit_index']
manufacturers_new_orders_consumer_goods = leading_indicators['manufacturers_new_orders_consumer_goods']
manufacturers_new_orders_ex_aircraft = leading_indicators['manufacturers_new_orders_ex_aircraft']
ten_year_minus_federal_funds_rate_monthly = leading_indicators['ten_year_minus_federal_funds_rate_monthly']

# ISM New Order Index plots with custom standard deviations

#plot ism_new_order_index with plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=ism_new_order_index.index, y=ism_new_order_index, mode='lines', name='ISM New Order Index'))
#add hline at 50
fig.add_shape(type='line', x0=ism_new_order_index.index.min(), x1=ism_new_order_index.index.max(), y0=50, y1=50, line=dict(color='gray', width=1), name='50')
fig.add_annotation(x=ism_new_order_index.index.max(), y=50, text='50', showarrow=False, yshift=10, font=dict(color='gray'))

fig.update_layout(title='ISM New Order Index', xaxis_title='Date', yaxis_title='Value', template='plotly_dark')
fig.show()


#plot average_weekly_hours_manufacturing with plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=average_weekly_hours_manufacturing.index, y=average_weekly_hours_manufacturing, mode='lines', name='Average Weekly Hours Manufacturing'))    
fig.update_layout(title='Average Weekly Hours Manufacturing', xaxis_title='Date', yaxis_title='Value', template='plotly_dark')
fig.show()

#plot initial_claims with plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=initial_claims.index, y=initial_claims, mode='lines', name='Initial Claims'))
fig.update_layout(title='Initial Claims', xaxis_title='Date', yaxis_title='Value', template='plotly_dark')
fig.show()

#PLot leading_credit_index with plotly
fig = go.Figure()
#plot hline at 0
fig.add_shape(type='line', x0=leading_credit_index.index.min(), x1=leading_credit_index.index.max(), y0=0, y1=0, line=dict(color='gray', width=1), name='0')
fig.add_annotation(x=leading_credit_index.index.max(), y=0, text='0', showarrow=False, yshift=10, font=dict(color='gray'))
#plot median line
fig.add_shape(type='line', x0=leading_credit_index.index.min(), x1=leading_credit_index.index.max(), y0=leading_credit_index.median(), y1=leading_credit_index.median(), line=dict(color='yellow', width=1), name='Median')
fig.add_annotation(x=leading_credit_index.index.max(), y=leading_credit_index.median(), text=f'Median: {leading_credit_index.median():.2f}', showarrow=False, yshift=10, font=dict(color='yellow'))
fig.add_trace(go.Scatter(x=leading_credit_index.index, y=leading_credit_index, mode='lines', name='Leading Credit Index'))
fig.update_layout(title='Leading Credit Index', xaxis_title='Date', yaxis_title='Value', template='plotly_dark')
fig.show()

#plot manufacturers_new_orders_consumer_goods with plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=manufacturers_new_orders_consumer_goods.index, y=manufacturers_new_orders_consumer_goods, mode='lines', name='Manufacturers New Orders Consumer Goods'))
fig.update_layout(title='Manufacturers New Orders Consumer Goods', xaxis_title='Date', yaxis_title='Value', template='plotly_dark')
fig.show()


#plot manufacturers_new_orders_ex_aircraft with plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=manufacturers_new_orders_ex_aircraft.index, y=manufacturers_new_orders_ex_aircraft, mode='lines', name='Manufacturers New Orders Ex Aircraft'))
fig.update_layout(title='Manufacturers New Orders Ex Aircraft', xaxis_title='Date', yaxis_title='Value', template='plotly_dark')
fig.show()


#plot ten_year_minus_federal_funds_rate_monthly with plotly
fig = go.Figure()
#plot hline at 2
fig.add_shape(type='line', x0=ten_year_minus_federal_funds_rate_monthly.index.min(), x1=ten_year_minus_federal_funds_rate_monthly.index.max(), y0=2, y1=2, line=dict(color='gray', width=1), name='2')
fig.add_annotation(x=ten_year_minus_federal_funds_rate_monthly.index.max(), y=2, text='2', showarrow=False, yshift=10, font=dict(color='gray'))

fig.add_trace(go.Scatter(x=ten_year_minus_federal_funds_rate_monthly.index, y=ten_year_minus_federal_funds_rate_monthly, mode='lines', name='Ten Year Minus Federal Funds Rate Monthly'))
fig.update_layout(title='Ten Year Minus Federal Funds Rate Monthly', xaxis_title='Date', yaxis_title='Value', template='plotly_dark')
fig.show()



In [ ]:
#Coincident Indicators

In [ ]:
data = pd.DataFrame()#ed.get_gdp_data()['real gdp']
data['GDP Growth'] = ed.get_gdp_data()['real gdp'].pct_change() * 100
# Dropping any NaN values resulting from the percentage change calculation

# Plotting the GDP growth
fig = px.line(data, title='GDP Growth Over Time')
fig.show()
data